In [1]:
import cirq

size = 3
qubits = cirq.GridQubit.square(size)
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [3]:
circuit = cirq.Circuit()

for q in qubits:
    if (q.row + q.col) % 2 == 0:
        circuit.append(cirq.H(q))
    else:
        circuit.append(cirq.X(q))

print(circuit)

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [4]:
#I HAVE NO IDEA WHAT AN ANSATZ IS (not entirely true, but still)

def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square grid of given length."""

    # Define the gate once and then re-use it for each Operation.
    rot = cirq.XPowGate(exponent=half_turns)

    # Create an X rotation Operation for each qubit in the grid.
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))

# Create the circuit using the rot_x_layer generator
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)


(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


In [5]:
import random
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length - 1, length)
    # links within a column
    jc = rand2d(length, length - 1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)
print('transverse fields: {}'.format(h))
print('row j fields: {}'.format(jr))
print('column j fields: {}'.format(jc))

transverse fields: [[-1, 1, -1], [1, 1, 1], [-1, -1, -1]]
row j fields: [[1, 1, 1], [1, -1, -1]]
column j fields: [[-1, 1], [1, 1], [-1, -1]]


In [6]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [7]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    cz_gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i + 1, j)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jr_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            q = cirq.GridQubit(i, j)
            q_1 = cirq.GridQubit(i, j + 1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)
            yield cz_gate(q, q_1)
            if jc_ij == -1:
                yield cirq.X(q)
                yield cirq.X(q_1)

In [8]:
def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    length = len(h)
    yield rot_x_layer(length, x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)

h, jr, jc = random_instance(3)

circuit = cirq.Circuit()    
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
               strategy=cirq.InsertStrategy.EARLIEST)
print(circuit)

                   ┌──────────┐   ┌──────────┐   ┌──────────┐                   ┌──────┐
(0, 0): ───X^0.1───────────────────@──────────────X─────────────@───────X──────────────────────────────────
                                   │                            │
(0, 1): ───X^0.1─────────@─────────┼────X───────────────────────@^0.3───X────────X─────────@───────X───────
                         │         │                                                       │
(0, 2): ───X^0.1────X────┼─────────┼──────────────@─────────────X───────X──────────────────@^0.3───X───────
                         │         │              │
(1, 0): ───X^0.1────Z^0.2┼─────────@^0.3──────────┼────@────────@──────────────────────────────────────────
                         │                        │    │        │
(1, 1): ───X^0.1─────────@^0.3──────────@─────────┼────┼────────@^0.3──────────────────────────────@───────
                                        │         │    │                                      

In [19]:
simulator = cirq.Simulator()
circuit = cirq.Circuit()    
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({0: 81, 2: 4, 256: 3, 4: 3, 8: 2, 16: 2, 32: 2, 64: 1, 128: 1, 1: 1})


In [20]:
import numpy as np

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy
print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

Counter({-3: 87, -1: 7, -9: 2, -11: 2, 3: 1, 1: 1})


In [21]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
print('Value of the objective function {}'.format(obj_func(results)))

Value of the objective function -3.04


In [22]:
import sympy
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))
print(circuit)

                     ┌─────────────┐   ┌──────────────┐   ┌──────────────┐                       ┌────────┐
(0, 0): ───X^alpha──────────────────────@──────────────────X─────────────────@─────────X──────────────────────────────────────────M('x')───
                                        │                                    │                                                    │
(0, 1): ───X^alpha──────────@───────────┼──────X─────────────────────────────@^gamma───X──────────X───────────@─────────X─────────M────────
                            │           │                                                                     │                   │
(0, 2): ───X^alpha────X─────┼───────────┼──────────────────@─────────────────X─────────X──────────────────────@^gamma───X─────────M────────
                            │           │                  │                                                                      │
(1, 0): ───X^alpha────Z^beta┼───────────@^gamma────────────┼──────@─────────

In [23]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)

In [24]:
sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)

         * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
for result in results:
    print(result.params.param_dict, obj_func(result))


OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.1)]) -2.86
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.30000000000000004)]) -2.72
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.5)]) -3.0
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.7000000000000001)]) -2.98
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.9)]) -2.84
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.1)]) -3.18
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) -2.52
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.5)]) -3.16
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) -2.82
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.9)]) -2.82
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.1)]) -3.2
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.30000000000000004)]) -2.8
OrderedDict([('alpha', 0.1), ('beta', 0.5

In [25]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=sweep_size)

         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=sweep_size)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=sweep_size))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print('Minimum objective value is {}.'.format(min))

Minimum objective value is -3.66.
